In [ ]:
import sys
sys.path.append("D:/ASOML/SNOCONE")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import os
from datetime import datetime
from CNN_memoryOptimization import *
from CNN_preProcessing import *
from CNN_benchmarks import *
from CNN_modelArchitectureBlocks import *
from CNN_FeatureImportance import*
print('modules imported')

# paths
domain = "Rockies"
WorkspaceBase = f"D:/ASOML/{domain}/"
modelWorkspace = f"D:/ASOML/{domain}/modelOutputs/fromAlpine/"
iteration = "20250717_073937"
phv_features = WorkspaceBase + "features/scaled/"
tree_workspace = WorkspaceBase + "treeCover/"
land_workspace = WorkspaceBase + "landCover/"
DMFSCAWorkspace = WorkspaceBase + "Rockies_DMFSCA/"
final_activation = 'relu'
architecture = 'Baseline'
featureTracking = f"{modelWorkspace}/{domain}_model_featureList_summary.csv"
print('paths set')

# set model specfic paths 
weights_path = f"{modelWorkspace}/{iteration}/best_weights_{iteration}.h5"
start_year = 2023
end_year = 2024

# create folder for model outputs
os.makedirs(f"{modelWorkspace}/{iteration}/shap_outputs/", exist_ok=True)
shap_outputs = f"{modelWorkspace}/{iteration}/shap_outputs/"

# loss function parameters
penalty_weight = 1
penalties_used = ["fSCA", "LearningRateScaling"]
low_snow_weight=0   
swe_threshold=0.01
fsca_threshold=0.1
low_threshold=0.05
penalty_scale= 0
learningRateTesting = 1e-3

# load in feature csv
feat_df = pd.read_csv(featureTracking)
feat_names = feat_df[[f'{iteration}']].dropna().astype(str)
featNo = len(feat_df[[f'{iteration}']].dropna().astype(str))
feature_names = feat_names[f'{iteration}'].dropna().astype(str).tolist()
print(feature_names)
print(featNo)

# get test data
X, y, featureNames = target_feature_stacks_SHAP(start_year=start_year, 
                                               end_year=end_year, 
                                               WorkspaceBase=WorkspaceBase, 
                                               ext = "nonull_fnl.tif", 
                                               vegetation_path = tree_workspace, 
                                               landCover_path = land_workspace, 
                                               phv_path = phv_features , 
                                               target_shape=(256,256),
                                               shapeChecks="Y",
                                               desired_features=feature_names,
                                               expected_channels=featNo)

# load in custom loss functions
custom_loss_fn = make_swe_fsca_loss(
        base_loss_fn=MeanSquaredError(),
        penalty_weight=penalty_weight,
        swe_threshold=swe_threshold,
        fsca_threshold=fsca_threshold,
        mask_value=-1
    )

custom_objects = {
    'make_swe_fsca_loss': make_swe_fsca_loss,
    'masked_rmse': masked_rmse,
    'masked_mae': masked_mae,
    'masked_mse': masked_mse,
    'LearningRateLogger': LearningRateLogger
}

# try Perm XAI
results = analyze_feature_importance(
        model_path=weights_path,
        X_test=X,
        y_test=y,
        custom_loss_function=custom_loss_fn,
        feature_names=feature_names,
        custom_objects=custom_objects,
        architecture=architecture,  
        featNo=featNo,
        final_activation=final_activation,
        n_repeats=10
    )

modules imported
paths set
['fSCA', 'DMFSCA', 'DOY', 'Tree Density', 'LandCover', 'ASO_CON_PlanCurv_albn83_60m_scl', 'ASO_CON_aspect_albn83_60m_scl', 'ASO_CON_DaH_albn83_60m_scl', 'ASO_CON_TRI_albn83_60m_scl', 'ASO_CON_VRM_albn83_60m_scl', 'ASOML_CON_windScour_60_albn83_scl', 'ASO_CON_gradMag_60_albn83_scl', 'ASO_CON_dem_albn83_60m_scl', 'ASO_CON_slope_albn83_60m_scl', 'ASO_CON_TPI_albn83_60m_scl', 'ASO_CON_TRASP_albn83_60m_scl', 'ASO_CON_GausCurv_albn83_60m_scl', 'ASO_CON_ProCurv_albn83_60m_scl']
18
Processing year 2023
